# ***Tool Calling***
- `tool binding`: is the step where your register tools with your langChain model so that 
    - llm know that how many tools are available
    - each tools what it does
    - input schema

In [14]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
load_dotenv()

True

In [15]:
llm = ChatGroq(model_name="meta-llama/llama-4-scout-17b-16e-instruct")

## Creating Tools

In [17]:
@tool
def multiply(a: int, b: int) -> int:
    """Given 2 integer number a and b this tool return their product"""
    return a*b

In [18]:
print(multiply.args_schema.model_json_schema())

{'description': 'Given 2 integer number a and b this tool return their product', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Tool Binding with chat memory

In [20]:
## Memory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [21]:
llm_with_tools = llm.bind_tools([multiply])

In [55]:
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

llm_with_tools_with_message_history = RunnableWithMessageHistory(llm_with_tools, get_session_history)

In [25]:
config = {"configurable": {"session_id": "chat1"}}

In [27]:
response = llm_with_tools_with_message_history.invoke(
    [
        HumanMessage(content="Hello sir")
    ],
    config=config
)
response

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 212, 'total_tokens': 234, 'completion_time': 0.043243787, 'prompt_time': 0.007473449, 'queue_time': 0.045381562, 'total_time': 0.050717236}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc691ff1-dc9c-49ff-8959-ccd025bf791c-0', usage_metadata={'input_tokens': 212, 'output_tokens': 22, 'total_tokens': 234})

In [29]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello sir', additional_kwargs={}, response_metadata={}), AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 212, 'total_tokens': 234, 'completion_time': 0.043243787, 'prompt_time': 0.007473449, 'queue_time': 0.045381562, 'total_time': 0.050717236}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'stop', 'logprobs': None}, id='run-bc691ff1-dc9c-49ff-8959-ccd025bf791c-0', usage_metadata={'input_tokens': 212, 'output_tokens': 22, 'total_tokens': 234})])}

# ***Tool calling***
- `Tool calling` is process where the llm decides, during a conversation or task, that it needs to use a specific tool(function) - and generates a structured output with
    - the name of the tool
    - and the arguments to call it with
- The main magics is
    - The llm does not actually run the tool it just suggest the tool and the input arguments. the actual execution is handled by LangChain or Programmer

## tools use in the query

In [32]:
response = llm_with_tools.invoke("can you multiply the 4 with 5")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1hb8', 'function': {'arguments': '{"a": 4, "b": 5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 217, 'total_tokens': 237, 'completion_time': 0.039360991, 'prompt_time': 0.007575367, 'queue_time': 0.211975577, 'total_time': 0.046936358}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-80e4d31e-8bee-42fa-9ad1-91dcf1dae121-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 5}, 'id': 'call_1hb8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 217, 'output_tokens': 20, 'total_tokens': 237})

### see tool call did not give the answer its just provide tools name and output format

In [33]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 4, 'b': 5},
  'id': 'call_1hb8',
  'type': 'tool_call'}]

In [34]:
response.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 4, 'b': 5},
 'id': 'call_1hb8',
 'type': 'tool_call'}

## ***Tools Execution*** Now execution by Programmer

In [36]:
parameter = response.tool_calls[0]['args']
parameter

{'a': 4, 'b': 5}

In [37]:
multiply.invoke(
    parameter
)

20

## See the magics if you pass the all content in invoking time
### New Messages `ToolMessages`

In [38]:
multiply.invoke(
    {'name': 'multiply',
    'args': {'a': 4, 'b': 5},
    'id': 'call_1hb8',
    'type': 'tool_call'}
)

ToolMessage(content='20', name='multiply', tool_call_id='call_1hb8')

## Lets do it at a time

In [56]:
store

{}

In [62]:
config = {"configurable": {"session_id": "chat3"}}

In [63]:
tool_messages_response = llm_with_tools_with_message_history.invoke(
    [
        HumanMessage(content='can you multiply 4 with 500')
    ],
    config=config
)
tool_messages_response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dc9n', 'function': {'arguments': '{"a": 4, "b": 500}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 216, 'total_tokens': 236, 'completion_time': 0.039348217, 'prompt_time': 0.007775421, 'queue_time': 0.044525185, 'total_time': 0.047123638}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3593f110-8881-40df-9eb9-fde6c17364a0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 500}, 'id': 'call_dc9n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 20, 'total_tokens': 236})

In [64]:
tool_messages_response.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 4, 'b': 500},
 'id': 'call_dc9n',
 'type': 'tool_call'}

In [ ]:
from langchain_core.messages import ToolMessage


In [65]:
llm_with_tools_with_message_history.invoke(
    [
        ToolMessage(
            tool_call_id=tool_messages_response.tool_calls[0]['id'], 
            content=tool_messages_response.tool_calls[0]
        )
    ],
    config=config
)

AIMessage(content='The result of multiplying 4 by 500 is 2000.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 279, 'total_tokens': 294, 'completion_time': 0.029606344, 'prompt_time': 0.009351707, 'queue_time': 0.048006133, 'total_time': 0.038958051}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'stop', 'logprobs': None}, id='run-e5942f33-4866-4924-8b19-7a8eb7f85adc-0', usage_metadata={'input_tokens': 279, 'output_tokens': 15, 'total_tokens': 294})

In [67]:
print(store['chat3'])

Human: can you multiply 4 with 500
AI: 
Tool: {'name': 'multiply', 'args': {'a': 4, 'b': 500}, 'id': 'call_dc9n', 'type': 'tool_call'}
AI: The result of multiplying 4 by 500 is 2000.


## Do it one more way without chat history

In [68]:
query = HumanMessage(content="Can you multiply 4 with 9")

In [69]:
messages_history = [query]
messages_history

[HumanMessage(content='Can you multiply 4 with 9', additional_kwargs={}, response_metadata={})]

In [72]:
result = llm_with_tools.invoke(
    "can you multiply 4 with 9"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wr2q', 'function': {'arguments': '{"a": 4, "b": 9}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 216, 'total_tokens': 236, 'completion_time': 0.039329473, 'prompt_time': 0.007665831, 'queue_time': 0.045742576, 'total_time': 0.046995304}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-16920fec-db8c-434b-a154-bfa5de81e1a1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 9}, 'id': 'call_wr2q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 20, 'total_tokens': 236})

In [73]:
messages_history.append(result)
messages_history

[HumanMessage(content='Can you multiply 4 with 9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wr2q', 'function': {'arguments': '{"a": 4, "b": 9}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 216, 'total_tokens': 236, 'completion_time': 0.039329473, 'prompt_time': 0.007665831, 'queue_time': 0.045742576, 'total_time': 0.046995304}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-16920fec-db8c-434b-a154-bfa5de81e1a1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 9}, 'id': 'call_wr2q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 20, 'total_tokens': 236})]

## invoking the tools

In [74]:
tool_result = multiply.invoke(
    result.tool_calls[0]
)
tool_result

ToolMessage(content='36', name='multiply', tool_call_id='call_wr2q')

In [75]:
messages_history.append(tool_result)
messages_history

[HumanMessage(content='Can you multiply 4 with 9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wr2q', 'function': {'arguments': '{"a": 4, "b": 9}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 216, 'total_tokens': 236, 'completion_time': 0.039329473, 'prompt_time': 0.007665831, 'queue_time': 0.045742576, 'total_time': 0.046995304}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-16920fec-db8c-434b-a154-bfa5de81e1a1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 4, 'b': 9}, 'id': 'call_wr2q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 216, 'output_tokens': 20, 'total_tokens': 236}),
 ToolMessage(content='36', name='multiply', tool_call_id='call_wr2q')]

In [76]:
llm_with_tools.invoke(messages_history)

AIMessage(content='The result of multiplying 4 and 9 is 36.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 245, 'total_tokens': 259, 'completion_time': 0.028443579, 'prompt_time': 0.00914667, 'queue_time': 0.044668427, 'total_time': 0.037590249}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f758b2a-9caa-4aaa-b383-0f6e39799977-0', usage_metadata={'input_tokens': 245, 'output_tokens': 14, 'total_tokens': 259})